# Group Assignment

### Import the necessary libraries

In [6]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime as dt

## PART I

## Examination of the January Effect
### The January Effect's main hypothesis states that stock returns are, on average, higher during January than during other months.

### 1. Download pricing data from Yahoo!Finance for each of the stocks in the ticker_list below and compute the continuously compounded daily returns using the Adjusted Closing price. Store these daily returns in a DataFrame called returns. Label the columns as 'RET_ticker', where ticker represents the trading symbol of the stock whose returns are stored in that column.

In [16]:
ticker_list = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']

In [17]:
yahoo = web.DataReader(ticker_list, 'yahoo')


In [18]:
yahoo.head()

Attributes   Adj Close                                               \
Symbols           AAPL        AXP          BA        CAT       CSCO   
Date                                                                  
2015-03-19  117.333099  75.381355  134.582214  67.653015  24.135815   
2015-03-20  115.860680  76.453728  134.922791  68.582230  24.289543   
2015-03-23  117.066200  75.991508  133.446930  68.497742  24.263922   
2015-03-24  116.587677  75.224182  132.433884  67.982475  23.913759   
2015-03-25  113.541618  74.059357  129.447250  67.458748  23.452562   

Attributes                                                             ...  \
Symbols           CVX         DIS          GS          HD         IBM  ...   
Date                                                                   ...   
2015-03-19  85.214813  100.070274  175.698196  102.699158  128.341202  ...   
2015-03-20  87.069588  101.058205  178.620819  104.777061  130.806686  ...   
2015-03-23  86.166611  100.862495  177.538681  103.510704  132.212097  ...   
2015-03-24  84.767357   99.827957  176.909760  103.582047  130.903015  ...   
2015-03-25  85.963242   97.861412  173.275009  101.379303  127.851295  ...   

Attributes      Volume                                                \
Symbols            NKE         PFE          PG        TRV        UNH   
Date                                                                   
2015-03-19  13536000.0  20437400.0   9257500.0  1448500.0  4032500.0   
2015-03-20  29779800.0  59160700.0  22580300.0  4366100.0  7195700.0   
2015-03-23  10471800.0  50806000.0   9891100.0  1535500.0  4576500.0   
2015-03-24   7598800.0  33163100.0   7478800.0  1217000.0  3054700.0   
2015-03-25   8133400.0  29206100.0   8482200.0  1663600.0  3385500.0   

Attributes                                                             
Symbols           UTX           V          VZ         WMT         XOM  
Date                                                                   
2015-03-19  2880800.0  15407500.0  13625000.0   7212900.0  12738500.0  
2015-03-20  8207000.0  15372000.0  26280800.0  12365300.0  36083500.0  
2015-03-23  4542800.0   8128000.0  13131700.0   6505700.0  17450700.0  
2015-03-24  4033800.0   6534800.0  11419600.0   5359100.0  10174300.0  
2015-03-25  4456500.0  13054300.0  14915500.0   6645900.0  11934700.0  

[5 rows x 168 columns]

In [30]:
adj_close = yahoo['Adj Close']
tickers = adj_close.columns

In [34]:
returns = pd.DataFrame() 
for ticker in tickers:
    returns['RET_'+ticker]= np.log(adj_close[ticker]/adj_close[ticker].shift(1))

    
returns # return for each ticker 

,RET_AAPL,RET_AXP,RET_BA,RET_CAT,RET_CSCO,RET_CVX,RET_DIS,RET_GS,RET_HD,RET_IBM,...,RET_NKE,RET_PFE,RET_PG,RET_TRV,RET_UNH,RET_UTX,RET_V,RET_VZ,RET_WMT,RET_XOM
Date,,,,,,,,,,,,,,,,,,,,,
2015-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-20,-0.012628,0.014126,0.002527,0.013642,0.006349,0.021532,0.009824,0.016497,0.020031,0.019028,...,0.036549,0.002924,0.016179,0.005666,-0.013506,0.000670,0.008940,0.005260,0.020880,0.001539
2015-03-23,0.010351,-0.006064,-0.010999,-0.001233,-0.001055,-0.010425,-0.001938,-0.006077,-0.012160,0.010687,...,-0.005112,0.023089,0.001415,-0.001003,-0.000840,-0.006551,-0.004907,0.001613,0.000841,0.010472
2015-03-24,-0.004096,-0.010149,-0.007620,-0.007551,-0.014537,-0.016372,-0.010310,-0.003549,0.000689,-0.009951,...,-0.008015,-0.015237,-0.011139,-0.003564,-0.007759,0.002020,0.000298,-0.004644,-0.003126,-0.010709
2015-03-25,-0.026474,-0.015606,-0.022810,-0.007734,-0.019474,0.014009,-0.019896,-0.020760,-0.021495,-0.023589,...,-0.016630,-0.006393,-0.010903,-0.014942,-0.013466,-0.020903,-0.020628,-0.013858,-0.021051,0.004014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-11,-0.035348,-0.079769,-0.200291,-0.056005,-0.078609,-0.023214,-0.054860,-0.069981,-0.051763,-0.056042,...,-0.049923,-0.072530,-0.077233,-0.060060,-0.007960,-0.104644,-0.054295,-0.030357,-0.045777,-0.033497
2020-03-12,-0.103978,-0.132663,-0.199778,-0.087436,-0.109718,-0.085053,-0.139085,-0.131696,-0.110651,-0.137548,...,-0.124053,-0.069170,-0.091428,-0.097106,-0.103730,-0.047721,-0.077329,-0.068498,-0.095092,-0.121422
2020-03-13,0.113157,0.175956,0.094582,0.076953,0.125517,0.089739,0.110337,0.161951,0.076148,0.048786,...,0.018428,0.085817,0.113409,0.050290,0.087172,0.027900,0.093844,0.056388,0.092204,0.024968


### 2. Compute the monthly returns and store them in a DataFrame named returns_mon. (*Use the resample() method and the rule = 'BM' - Business Month End to compute the monthly returns*). 

### 3. Create a dummy variable (labeled "JAN_DUMMY") in the DataFrame returns_mon, and assign it a value of 1 if the month is January, and 0 otherwise.

### 4. Use the DataFrame returns_mon to compute the average returns in January, and during non-January months for each year and for each stock, and store them in a new DataFrame named returns_jan_mon. 

### 5. a. Use the DataFrame returns_mon_jan to test, for each stock, whether the average return in January is higher than the average return on non-January months. Store the t-tests and p-values in a Dataframe called results_stock, along with the average returns in January, and during non-January months for each stock. The heading for the DataFrame is found below:

In [45]:
results_stock.head(0)

,Avg_Jan,Avg_NonJan,T_Test,P_Value
Ticker,,,,


### 5. b. Use the DataFrame results_stock to output the stocks for which the January effect hypothesis is true with a 95% degree of confidence. What about the observations where the hypothesis is true with 90% degree of confidence.

### 6. a. Use the DataFrame returns_mon_jan to test, for each year, whether the average return in January is different from the average return on non-January months. Store the t-tests and p-values in a Dataframe called results_year, along with the average returns in January, and during non-January months for each year. The heading for the DataFrame is found below:

In [45]:
results_stock.head(0)

,Avg_Jan,Avg_NonJan,T_Test,P_Value
Ticker,,,,


### 6. b. Use the DataFrame results_year to output the observations where the January effect hypothesis is true with a 95% degree of confidence. What about the observations where the hypothesis is true with 90% degree of confidence.

### 7. Create a bar plot of the average return in January, as well as the average return across all non-January months, for each year in the sample.

### 8. Briefly explain whether the evidence is consistent with the January effect, and if not, name a couple of limitations of our dataset and/or approach.

### The following questions apply to a random sample of fifteen stocks that your group will select by running the following code 

### Random Stock Selection 

In [9]:
np.random.seed (1000 + # add here your group  number as listed on Blackboard)
ticker_list = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']
stock_lst = np.random.choice(ticker_list,15,replace=False)
stock_lst

## PART II

### 1. Modify the returns_mon DataFrame to store only the monthly return for the fifteen stocks designated for your group. 

### 2. For each stock, compute the cumulative return series over the entire sample (no reset), and store the series in a new DataFrame named sample_ret.

### 3a. Which stocks were the top three performers. What were their cumulative returns over the entire sample period?

### 3b. Which stocks were the worst three performers. What were their cumulative returns over the entire sample period?

### 3c. Create a line plot of the series for the stocks with the highest three and lowest three cumulative returns.

## PART III

### 1. Use the DataFrame returns_mon to create a new DataFrame named data_intial that only has the data from January 2013 to December 2017. In addition, create another DataFrame named data_post to store the 2018 and 2019 data.

### 2. Create a DataFrame named rf_rate to store the risk free rate over the period from January 2013 to December 2019. The risk-free rate  is available in the ff_factors19.csv file on Blackboard.

### 3a. Use the DataFrames data_initial and rf_rate, in conjunction with Scipy' s minimize() function to find the optimal portfolio (highest Sharpe ratio) from the universe of stocks comprised of all the stocks in your sample. (Assume there are no short-selling constraints).

### 3b. The weights of the stocks in the optimal portfolio are:

### 3c. The optimal portfolio return is:

### 3d. The optimal portfolio standard deviation is:

### 3e. The Sharpe Ratio the optimal portfolio is:

### 4a. What is the return of the Global Minimum Variance Portfolio (GMVP)

### 4b. What is the standard deviation of the Global Minimum Variance Portfolio (GMVP)

### 4c. What are the weights of the stocks in the Global Minimum Variance Portfolio (GMVP)

### 5. Use Scipy's minimize() function to solve for the efficient frontier. In addition, plot it, add to the plot the fifteen stocks, the GMVP, the optimal portfolio, as well as the Capital Allocation Line.

## PART IV

### 1. Create a DataFrame named weights with three columns: 1) TICKER; 2) Optimal_Weights and 3) Equal_Weights for each ticker.

### 2. Using the DataFrames data_post and weights, compute the two portfolio (Optimal and Equal-weighted) monthy returns over the period January 1st, 2018 and December 31st, 2019. Store the portfolio returns into a new DataFrame named port_returns. Label the two columns as 'Optimal Portfolio' and 'EW_Portfolio'

### 3. In a single output, provide summary statistics on the two portfolios.

### 4a. Compute the two-year cumulative return series for both portfolios and plot them in the same graph. 

### 4b. What was the two-year cumulative return for the best performer? What was the two-year cumulative return for the worst performer?